In [7]:
import os
import gc
import csv
import scipy.io
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [8]:
class Basics:
    def __init__(self):
        self.getcwd = os.getcwd()
        self.dotcsv = ".csv"

    def Get_path_files(self, subfolder1, subfolder2):
        path_folder = self.getcwd.split("src")[0] + f"data/inputs/{subfolder1}/{subfolder2}_"
        path_files = [(path_folder + f"{i}" + self.dotcsv) for i in range(1, 366)]
        return path_files

In [9]:
basics = Basics()

In [10]:
def Get_gen_data(basics=basics):
    data_raw_gen = scipy.io.loadmat(f"{basics.getcwd}/../data/inputs/KPG193_ver1_2.mat")
    gen_count = 122

    # generator data - ordered by generatior id
    # unit: MW
    # ignoring status; following list is the number of hours in the year for the 15 killed generators to be committed again in the year
    # [47, 4, 10, 45, 576, 744, 4440, 7416, 3336, 14, 92, 32, 3, 315, 45]
    # it was included in the UC at least; enough reason to ignore status
    gen_data = data_raw_gen["mpc"][0, 0]["gen"][:, [8, 9]]
    gen_pmax = gen_data[:, 0]
    gen_pmin = gen_data[:, 1]

    # generator cost function - ordered by generatior id
    # unit: 1,000 KRW and MWh
    gencost_data = data_raw_gen["mpc"][0, 0]["gencost"][:, 4:]
    gencost_c2 = gencost_data[:, 0]
    gencost_c1 = gencost_data[:, 1]
    gencost_c0 = gencost_data[:, 2]

    # generator unit types - ordered by generatior id
    gentypes = np.array([0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
    gentypes_description = {0: "Coal", 1: "LNG", 2: "Nuclear"}
    gentypes_colors = np.array([[1, 0, 0], [0, 0, 1], [0, 1, 0]])[gentypes] # red; blue; green
    idx_gen_coal = np.where(gentypes == 0)[0]
    idx_gen_lng = np.where(gentypes == 1)[0]
    idx_gen_nuclear = np.where(gentypes == 2)[0]

    return gen_count, gen_pmax, gen_pmin, gencost_c2, gencost_c1, gencost_c0, gentypes, gentypes_description, gentypes_colors, idx_gen_coal, idx_gen_lng, idx_gen_nuclear, 

In [11]:
gen_count, gen_pmax, gen_pmin, gencost_c2, gencost_c1, gencost_c0, gentypes, gentypes_description, gentypes_colors, idx_gen_coal, idx_gen_lng, idx_gen_nuclear = Get_gen_data()

In [12]:
def Get_demands(basics=basics):
    demands = []
    for path_file in basics.Get_path_files("demand", "daily_demand"):
        with open(path_file) as csvfile:
            data = np.array([row for row in csv.reader(csvfile)])[1:, :-1].astype(float) # len(data) == 197 * 24 == 4728 for all csv files
            demands.append([data[data[:, 0] == hour][:, -1].sum() for hour in range(1, 25)])

    return np.array(demands).reshape(-1) # demands; len(demands) == 365 * 24 == 8760

In [13]:
total_demands = Get_demands() 

In [14]:
def Get_renewable_capacity(basics=basics):
    with open(basics.getcwd.split("src")[0] + "data/inputs/renewables_capacity/solar_generators_2022.csv") as f:
        data_raw_solar = np.array([row for row in csv.reader(f)])[1:, [0, 2]].astype(float)
    data_solar = np.zeros(197)
    data_solar[data_raw_solar[:, 0].astype(int) - 1] = data_raw_solar[:, 1] # missing 4 bus id covered with 0 (bus number unified as 197 as indicated in the demand)

    with open(basics.getcwd.split("src")[0] + f"data/inputs/renewables_capacity/wind_generators_2022.csv") as csvfile:
        data_raw_wind = np.array([row for row in csv.reader(csvfile)])[1:, [0, 2]].astype(float) # bus id ordered pmax, pmin is all 0; ignore
    data_wind = np.zeros(197)
    data_wind[data_raw_wind[:, 0].astype(int) - 1] = data_raw_wind[:, 1] # missing 4 bus id covered with 0 (bus number unified as 197 as indicated in the demand)

    with open(basics.getcwd.split("src")[0] + f"data/inputs/renewables_capacity/hydro_generators_2022.csv") as csvfile:
        data_hydro = np.array([row for row in csv.reader(csvfile)])[1:, 2].astype(float)

    return np.column_stack([data_solar, data_wind, data_hydro]) # renewable_capacity

In [15]:
renewable_capacity = Get_renewable_capacity()

In [16]:
def Get_renewable_generation(basics=basics, renewable_capacity=renewable_capacity):
    renewable_capacity_24_repeat = np.repeat(renewable_capacity, 24, axis=0)
    renewable_generation = np.zeros(24 * 365)
    idx_hour = 0

    for path_file in basics.Get_path_files("renewables", "renewables"): # hour, busid, solar, wind, hydro
        # DAILY PROFILE RATIO
        with open(path_file) as csvfile:
            reader = csv.reader(csvfile)
            next(reader)
            data_raw_reg_ratio = np.array([[float(cell.strip()) if cell.strip() else 0.0 for cell in row] for row in reader]) # some bad rows (pv ratio has nothing) - covererd
            # H1, BUS1 / H2, BUS1 / H3, BUS3/ ...
            # just really messy code but logic is correct and gpt doesn't work only 5 sec and correct hourly reg...
            # also i might do something with REG for ppt images and this loop structure favors future amandments but i have 36hrs left
            reg_per_hour_bus = (data_raw_reg_ratio[:, 2:] * renewable_capacity_24_repeat).sum(axis=1)
            hours = data_raw_reg_ratio[:, 0]
            for hour_incremental in range(1, 25):
                for hour, reg in zip(hours, reg_per_hour_bus):
                    if hour == hour_incremental:
                        renewable_generation[idx_hour] += reg
                idx_hour += 1

    return renewable_generation

In [17]:
renewable_generation = Get_renewable_generation()
demands = total_demands - renewable_generation # demands is now hourly demand to be fulfilled by thermal generators

In [28]:
def Get_commitmnets_matrix(basics=basics):
    matrix_for_zeroindexing = np.stack((np.ones(2928), np.ones(2928), np.zeros(2928)), axis=1).astype(int) # for data from the whole csv file
    commitments_matrix = np.zeros((365 * 24, 122), dtype=bool)

    for day, path_file in enumerate(basics.Get_path_files("commitment_decision", "commitment_decision")):
        # DAILY
        with open(path_file) as csvfile:
            reader = csv.reader(csvfile)
            data = np.array([row for row in reader][1:], dtype=int) # all correct len
            data -= matrix_for_zeroindexing # idk if this is still needed zeroindexing of busid

        for hour in range(0, 24): # 0-index hour
            commitments_matrix[(day * 24 + hour)] = data[(122 * hour):(122 * (hour + 1))][:, -1].reshape(-1).astype(bool) # checked by another primitive method
    
    return commitments_matrix

In [29]:
commitments_matrix = Get_commitmnets_matrix()

In [30]:
commitments_matrix

array([[ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True, False,  True],
       ...,
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True, False,  True]])

---
---
---

In [ ]:
def Compute_smp(idx_hour):
    status = 0
    commitments = commitments_matrix[idx_hour]

    # so sorry for the garbage codde 
    gen_count_smp = commitments.sum()
    gen_pmax_smp = gen_pmax[commitments]
    gen_pmin_smp = gen_pmin[commitments]
    gencost_c2_smp = gencost_c2[commitments]
    gencost_c1_smp = gencost_c1[commitments]
    gencost_c0_smp = gencost_c0[commitments]

    demand = demands[idx_hour] # MWh
    if not (demand >= gen_pmin_smp.sum()):
        status = -1
        return status, np.nan, np.nan
    if not (demand <= gen_pmax_smp.sum()):
        status = -2
        return status, np.nan, np.nan


    gen_power = cp.Variable(gen_count_smp)
    objective_function = cp.sum(cp.multiply(gencost_c2_smp, cp.square(gen_power)) + cp.multiply(gencost_c1_smp, gen_power) + gencost_c0_smp)
    objective = cp.Minimize(objective_function)
    constraints = [
        gen_power >= gen_pmin_smp, 
        gen_power <= gen_pmax_smp,
        cp.sum(gen_power) == demand,
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.OSQP)


    smp = -constraints[-1].dual_value # 1 $ / kWh = 1000 krw / kWh (according to m file from KPG)
    error_p_equality_constraint = abs(gen_power.value.sum() - demand) / demand * 100

    return status, smp, error_p_equality_constraint

In [ ]:
results = []
for idx_hour in range(0, 8760): # 365 * 24
    results.append(Compute_smp(idx_hour))
results = np.array(results) # 183 status -1, 8570 out of 8760 were feasible